# install dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Projects/CSE_583'

/content/drive/MyDrive/Projects/CSE_583


In [ ]:
!pip install ellipsis
!pip install h3pandas
!pip install -y h3-py
!pip install geojson
import h3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
  Created wheel for pygltflib: filename=pygltflib-1.16.1-py3-none-any.whl size=27086 sha256=9f538d290e01d21ae526f46b6bc162d78a3e12444572e4826f163cf3edacca8b
  Stored in directory: /root/.cache/pip/wheels/9c/73/91/ae3a93bb6cc6dde662ed1dc48dd0ec7ca41a7bcc2a1a99b5a6
Successfully built pygltflib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.8

In [ ]:
!pip install mercantile
!pip install folium
!pip install rasterio
!pip install rasterstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
from shapely.geometry import Polygon
from geojson import Feature, Point, FeatureCollection
import json
import math
import mercantile
from tqdm import tqdm
from folium import GeoJson
import os
import tempfile
import fiona
import folium
import os
import requests
import rasterio
import rasterstats
import matplotlib
import matplotlib.pyplot as plt

# 0: file management

## file for this notebook

In [ ]:
raw_folder = 'data/raw/'
raw_building_fn = "example_building_footprint.geojson"
raw_weather_fn = "example_weather.csv"
raw_terrain_fn = "example_dem.tif"
raw_extent_fn = "example_aoi_polygon.geojson"
raw_boundary_fn = "example_area_boundary.geojson"

In [ ]:
raw_building_path = os.path.join(raw_folder, raw_building_fn)
raw_extent_path = os.path.join(raw_folder, raw_extent_fn)
raw_weather_path = os.path.join(raw_folder, raw_weather_fn)
raw_terrain_path = os.path.join(raw_folder, raw_terrain_fn)
raw_boundary_path = os.path.join(raw_folder, raw_boundary_fn)
print(raw_building_path)
print(raw_extent_path)
print(raw_weather_path)
print(raw_terrain_path)
print(raw_boundary_path)

data/raw/example_building_footprint.geojson
data/raw/example_aoi_polygon.geojson
data/raw/example_weather.csv
data/raw/example_dem.tif
data/raw/example_area_boundary.geojson


## file from this notebook

In [ ]:
aggr_hex_fn = "example_aggr_hexagon.geojson"
aggr_hex_path = os.path.join(raw_folder, aggr_hex_fn)

# 1: read raw file for building, weather, terrain, and extent. align all to 4326

In [ ]:
# with fiona.open(raw_terrain_path) as src:
#     terrain_data = src.read()
with rasterio.open(raw_terrain_path) as src:
    raw_terrain = src.read()
print(src.meta)
print(type(raw_terrain))
print(raw_terrain.shape)
# raw_terrain

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -999999.0, 'width': 2287, 'height': 2455, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(9.259259273673361e-05, 0.0, -122.3737962958602,
       0.0, -9.259259273673361e-05, 47.721203704191254)}
<class 'numpy.ndarray'>
(1, 2455, 2287)


In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling

input_file = raw_terrain_path
output_file = raw_terrain_path

# Open the input file
with rasterio.open(input_file) as src:
    # Define the new CRS
    dst_crs = 'EPSG:4326'

    # Calculate the transform and dimensions for the new CRS
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)

    # Create a metadata dictionary for the output file
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Open the output file
    with rasterio.open(output_file, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            # Reproject and write each band
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

In [ ]:
raw_building = gpd.read_file(raw_building_path)
raw_extent = gpd.read_file(raw_extent_path)
raw_weather = pd.read_csv(raw_weather_path)
raw_boundary = gpd.read_file(raw_boundary_path)

# 2: create hexagon for weather data, decide hexgon radius

In [ ]:
def create_hexagon(lat, lon, radius):
    # number of sides
    n_sides = 6
    # Convert latitude and longitude to radians
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    # Earth's radius in meters
    earth_radius = 6371000

    # Calculate the radius of our hexagon in radians
    r_rad = radius / earth_radius

    # Calculate the vertices of the hexagon
    hexagon_vertices = []
    for i in range(n_sides):
        angle = math.pi / 3 * i
        x = lon_rad + r_rad * math.cos(angle)
        y = lat_rad + r_rad * math.sin(angle)
        # Convert the vertex back to degrees
        vertex = (math.degrees(x), math.degrees(y))
        hexagon_vertices.append(vertex)

    return Polygon(hexagon_vertices)
def hex_to_geojson(hexagon):
    return GeoJson({
        "type": "Feature",
        "geometry": shapely.geometry.mapping(hexagon)
    })

In [ ]:
raw_extent

,geometry
0,"POLYGON ((-122.37375 47.72119, -122.37375 47.4..."


In [ ]:
rex_radius = 160
raw_weather['hex'] = raw_weather.apply(lambda row: create_hexagon(row['Lat'], row['Lon'], rex_radius), axis=1)
raw_weather.head()

,Station ID,Station Name,Lat,Lon,Elev (ft.),Ave temp annual_F,Note,hex
0,KWASEATT2119,Belltown - Lenora,47.613,-122.342,194,50.107945,NaN,"POLYGON ((-122.34056108543052 47.613, -122.341..."
1,KWASEATT2336,Avalon LQA Seattle,47.620,-122.356,120,53.560274,NaN,"POLYGON ((-122.35456108543053 47.62, -122.3552..."
2,KWASEATT2379,J'Oz 4 - The Veduta,47.621,-122.327,246,52.952329,NaN,POLYGON ((-122.32556108543052 47.6210000000000...
3,KWASEATT2850,mrweather Capitol Hill,47.622,-122.318,335,NaN,no data,"POLYGON ((-122.31656108543052 47.622, -122.317..."
4,KWASEATT2696,Seattle CD,47.602,-122.312,253,53.064187,NaN,"POLYGON ((-122.31056108543052 47.602, -122.311..."


## save te gepdataframe to raw_hex

*   Station ID, Lon, Lat, Temp, Elev
*   hex



In [ ]:
raw_hex = raw_weather.copy()[['hex','Station ID','Lat', 'Lon', 'Elev (ft.)', 'Ave temp annual_F','Note']]
raw_hex.head()

,hex,Station ID,Lat,Lon,Elev (ft.),Ave temp annual_F
0,"POLYGON ((-122.34056108543052 47.613, -122.341...",KWASEATT2119,47.613,-122.342,194,50.107945
1,"POLYGON ((-122.35456108543053 47.62, -122.3552...",KWASEATT2336,47.620,-122.356,120,53.560274
2,POLYGON ((-122.32556108543052 47.6210000000000...,KWASEATT2379,47.621,-122.327,246,52.952329
3,"POLYGON ((-122.31656108543052 47.622, -122.317...",KWASEATT2850,47.622,-122.318,335,NaN
4,"POLYGON ((-122.31056108543052 47.602, -122.311...",KWASEATT2696,47.602,-122.312,253,53.064187


In [ ]:
m = folium.Map(location=[raw_hex.iloc[0]['Lat'], raw_hex.iloc[0]['Lon']], zoom_start=14)
for _, row in raw_hex.iterrows():
    geojson_hex = hex_to_geojson(row['hex'])
    geojson_hex.add_to(m)
    # Add the point
    folium.Marker(
        [row['Lat'], row['Lon']],
        popup=f'Lat: {row["Lat"]}, Lon: {row["Lon"]}'
    ).add_to(m)
m

# 3: aggregate info to hexgon

In [ ]:
import numpy as np
def weighted_median(data, weights):
    data_sorted, weights_sorted = zip(*sorted(zip(data, weights)))
    cum_weights = np.cumsum(weights_sorted)
    cutoff = weights.sum() / 2.0
    return np.interp(cutoff, cum_weights, data_sorted)

def weighted_percentile(data, weights, percentile):
    data_sorted, weights_sorted = zip(*sorted(zip(data, weights)))
    cum_weights = np.cumsum(weights_sorted)
    cutoff = weights.sum() * percentile / 100.0
    return np.interp(cutoff, cum_weights, data_sorted)

def weighted_std(data, weights):
    average = np.average(data, weights=weights)
    variance = np.average((data - average)**2, weights=weights)
    return np.sqrt(variance)

## building

### get centroid of building geometry, select whatever building whose centroid is in the hexagon as aggr.

In [ ]:
# if raw_hex is not None:
#     del raw_hex

In [ ]:
raw_building['centroid'] = raw_building.geometry.centroid
raw_building.head()

<ipython-input-19-0654a740864c>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  raw_building['centroid'] = raw_building.geometry.centroid


,id,height,geometry,centroid
0,0,6.167338,"POLYGON ((-122.16323 47.51119, -122.16322 47.5...",POINT (-122.16334 47.51125)
1,1,6.117427,"POLYGON ((-122.16348 47.50285, -122.16367 47.5...",POINT (-122.16357 47.50269)
2,2,4.593605,"POLYGON ((-122.16416 47.50349, -122.16446 47.5...",POINT (-122.16434 47.50342)
3,3,3.569735,"POLYGON ((-122.16704 47.50321, -122.16705 47.5...",POINT (-122.16706 47.50322)
4,4,6.165513,"POLYGON ((-122.16833 47.50363, -122.16817 47.5...",POINT (-122.16825 47.50369)


In [ ]:
def average_building_height_with_centroid(buildings, hexagon):
    # Select buildings whose centroid is within or intersects the hexagon
    buildings_within_hex = buildings[buildings['centroid'].within(hexagon.hex)]
    # print(buildings_within_hex.columns)
    if buildings_within_hex.empty:
        # return (np.nan, np.nan, np.nan, np.nan, np.nan, np.nan)
        return {
            'centroid_stat_total_height_area': np.NaN,
            'centroid_stat_avg_height_area': np.NaN,
            'centroid_stat_mean': np.NaN,
            'centroid_stat_median': np.NaN,
            'centroid_stat_std_dev': np.NaN,
            'centroid_stat_min': np.NaN,
            'centroid_stat_25%': np.NaN,
            'centroid_stat_50%': np.NaN,
            'centroid_stat_75%': np.NaN,
            'centroid_stat_max': np.NaN
        }
    # Calculate the product of the area and height for each building
    buildings_within_hex['area_height'] = buildings_within_hex.area * buildings_within_hex['height']


    # Method 1: related to hexagon area
    # Sum the products and divide by the area of the hexagon to get the average height
    total_height_area = buildings_within_hex['area_height'].sum()
    hexagon_area = hexagon.hex.area
    print(hexagon_area)
    average_height_area = total_height_area / hexagon_area if hexagon_area != 0 else 0

    # Method 2: not related to hexagon area
    weighted_avg = np.average(buildings_within_hex['height'], weights=buildings_within_hex.area)

    median = weighted_median(buildings_within_hex['height'], buildings_within_hex.area)
    percentile_0 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 0)
    percentile_25 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 25)
    percentile_50 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 50)
    percentile_75 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 75)
    percentile_100 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 100)
    std_dev = weighted_std(buildings_within_hex['height'], buildings_within_hex.area)

    return {
        'centroid_stat_total_height_area': total_height_area,
        'centroid_stat_avg_height_area': average_height_area,
        'centroid_stat_mean': weighted_avg,
        'centroid_stat_median': median,
        'centroid_stat_std_dev': std_dev,
        'centroid_stat_min': percentile_0,
        'centroid_stat_25%': percentile_25,
        'centroid_stat_50%': percentile_50,
        'centroid_stat_75%': percentile_75,
        'centroid_stat_max': percentile_100
    }

In [ ]:
raw_hex['centroid_stat_ALL'] = raw_hex.apply(lambda x: average_building_height_with_centroid(raw_building, x), axis=1)


<ipython-input-20-caf2dfae4ba1>:20: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_within_hex['area_height'] = buildings_within_hex.area * buildings_within_hex['height']
<ipython-input-20-caf2dfae4ba1>:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  weighted_avg = np.average(buildings_within_hex['height'], weights=buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:33: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  median = weighted_median(buildings_within_hex['height'], buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:34: UserWarning: Geometry is in a geo

5.3792522028331555e-06


<ipython-input-20-caf2dfae4ba1>:36: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  percentile_50 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 50)
<ipython-input-20-caf2dfae4ba1>:37: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  percentile_75 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 75)
<ipython-input-20-caf2dfae4ba1>:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  percentile_100 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 100)
<ipython-input-20-caf2dfae4ba1>:39: UserWarn

5.3792522028508644e-06


<ipython-input-20-caf2dfae4ba1>:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  weighted_avg = np.average(buildings_within_hex['height'], weights=buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:33: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  median = weighted_median(buildings_within_hex['height'], buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:34: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  percentile_0 = weighted_percentile(buildings_within_hex['height'], buildings_within_hex.area, 0)
<ipython-input-20-caf2dfae4ba1>:35: UserWarning: Geometry is in a geo

...

5.3792522028331555e-06


<ipython-input-20-caf2dfae4ba1>:20: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_within_hex['area_height'] = buildings_within_hex.area * buildings_within_hex['height']
<ipython-input-20-caf2dfae4ba1>:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  weighted_avg = np.average(buildings_within_hex['height'], weights=buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:33: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  median = weighted_median(buildings_within_hex['height'], buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:34: UserWarning: Geometry is in a geo

5.379252202850864e-06


<ipython-input-20-caf2dfae4ba1>:20: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_within_hex['area_height'] = buildings_within_hex.area * buildings_within_hex['height']
<ipython-input-20-caf2dfae4ba1>:31: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  weighted_avg = np.average(buildings_within_hex['height'], weights=buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:33: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  median = weighted_median(buildings_within_hex['height'], buildings_within_hex.area)
<ipython-input-20-caf2dfae4ba1>:34: UserWarning: Geometry is in a geo

5.3792522028331555e-06


In [ ]:
for stat in ['total_height_area','avg_height_area','mean','median', 'std_dev', 'min', '25%','50%' ,'75%', 'max']:
    # try:
    #     raw_hex = raw_hex.drop(['centroid_stat_' + stat],axis = 1)
    # except KeyError:
    #     pass
    raw_hex['centroid_stat_' + stat] = raw_hex['centroid_stat_ALL'].apply(lambda x: x['centroid_stat_' + stat])

In [ ]:
raw_hex.columns

Index(['hex', 'Station ID', 'Lat', 'Lon', 'Elev (ft.)', 'Ave temp annual_F',
       'centroid_stat_ALL', 'intercestion_stat_ALL',
       'centroid_stat_total_height_area', 'centroid_stat_avg_height_area',
       'centroid_stat_mean', 'centroid_stat_median', 'centroid_stat_std_dev',
       'centroid_stat_min', 'centroid_stat_25%', 'centroid_stat_50%',
       'centroid_stat_75%', 'centroid_stat_max'],
      dtype='object')

### get whatever within a hexagon, including the interesction part

In [ ]:
def building_height_stats_intersection(buildings, hexagon):
    # 创建一个临时 DataFrame 来存储高度和权重
    temp_df = pd.DataFrame()
    total_height_area = 0
    # 遍历每个建筑物
    for idx, building in buildings.iterrows():
        intersection = building.geometry.intersection(hexagon.hex)
        if not intersection.is_empty:
            temp_df = temp_df.append({'height': building['height'], 'weight': intersection.area}, ignore_index=True)
            intersection_area = intersection.area
            building_height = building['height']
            total_height_area += intersection_area * building_height
    # Calculate the hexagon's area
    hexagon_area = hexagon.hex.area

    # Calculate the average height as the total area-height product divided by the hexagon area
    average_height_area = total_height_area / hexagon_area if hexagon_area != 0 else 0

    # 计算加权统计数据
    if not temp_df.empty:
        weighted_avg = np.average(temp_df['height'], weights=temp_df['weight'])
        median = weighted_median(temp_df['height'], temp_df['weight'])
        std_dev = weighted_std(temp_df['height'], temp_df['weight'])
        percentile_0 = weighted_percentile(temp_df['height'], temp_df['weight'], 0)
        percentile_25 = weighted_percentile(temp_df['height'], temp_df['weight'], 25)
        percentile_50 = weighted_percentile(temp_df['height'], temp_df['weight'], 50)
        percentile_75 = weighted_percentile(temp_df['height'], temp_df['weight'], 75)
        percentile_100 = weighted_percentile(temp_df['height'], temp_df['weight'], 100)
    else:
        total_height_area =average_height_area=weighted_avg = median = std_dev = percentile_0 = percentile_25=percentile_50 = percentile_75 = percentile_100 = np.nan


    return {
        'intercestion_stat_total_height_area': total_height_area,
        'intercestion_stat_avg_height_area': average_height_area,
        'intercestion_stat_mean': weighted_avg,
        'intercestion_stat_median': median,
        'intercestion_stat_std_dev': std_dev,
        'intercestion_stat_min': percentile_0,
        'intercestion_stat_25%': percentile_25,
        'intercestion_stat_50%': percentile_50,
        'intercestion_stat_75%': percentile_75,
        'intercestion_stat_max': percentile_100
    }

In [ ]:
raw_hex['intercestion_stat_ALL'] = raw_hex.apply(lambda x: building_height_stats_intersection(raw_building, x), axis=1)

Streaming output truncated to the last 5000 lines.
<ipython-input-22-56f34c7ae959>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append({'height': building['height'], 'weight': intersection.area}, ignore_index=True)
<ipython-input-22-56f34c7ae959>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append({'height': building['height'], 'weight': intersection.area}, ignore_index=True)
<ipython-input-22-56f34c7ae959>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append({'height': building['height'], 'weight': intersection.area}, ignore_index=True)
<ipython-input-22-56f34c7ae959>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a

In [ ]:
for stat in ['total_height_area','avg_height_area','mean','median', 'std_dev', 'min', '25%','50%' ,'75%', 'max']:
    # try:
    #     raw_hex = raw_hex.drop(['intercestion_stat_' + stat],axis = 1)
    # except KeyError:
    #     pass
    raw_hex['intercestion_stat_' + stat] = raw_hex['intercestion_stat_ALL'].apply(lambda x: x['intercestion_stat_' + stat])

In [ ]:
raw_hex.columns

Index(['hex', 'Station ID', 'Lat', 'Lon', 'Elev (ft.)', 'Ave temp annual_F',
       'centroid_stat_ALL', 'intercestion_stat_ALL',
       'centroid_stat_total_height_area', 'centroid_stat_avg_height_area',
       'centroid_stat_mean', 'centroid_stat_median', 'centroid_stat_std_dev',
       'centroid_stat_min', 'centroid_stat_25%', 'centroid_stat_50%',
       'centroid_stat_75%', 'centroid_stat_max',
       'intercestion_stat_total_height_area',
       'intercestion_stat_avg_height_area', 'intercestion_stat_mean',
       'intercestion_stat_median', 'intercestion_stat_std_dev',
       'intercestion_stat_min', 'intercestion_stat_25%',
       'intercestion_stat_50%', 'intercestion_stat_75%',
       'intercestion_stat_max'],
      dtype='object')

In [ ]:
raw_hex[['Station ID', 'Elev (ft.)',
          'centroid_stat_mean','centroid_stat_median','centroid_stat_std_dev', 'centroid_stat_total_height_area', 'centroid_stat_avg_height_area',
        #  'centroid_stat_min', 'centroid_stat_25%', 'centroid_stat_50%', 'centroid_stat_75%', 'centroid_stat_max',
          'intercestion_stat_mean','intercestion_stat_median', 'intercestion_stat_std_dev', 'intercestion_stat_total_height_area', 'intercestion_stat_avg_height_area',
    #    'intercestion_stat_min', 'intercestion_stat_25%', 'intercestion_stat_50%', 'intercestion_stat_75%','intercestion_stat_max'
       ]]

,Station ID,Elev (ft.),centroid_stat_mean,centroid_stat_median,centroid_stat_std_dev,centroid_stat_total_height_area,centroid_stat_avg_height_area,intercestion_stat_mean,intercestion_stat_median,intercestion_stat_std_dev,intercestion_stat_total_height_area,intercestion_stat_avg_height_area
0,KWASEATT2119,194,18.714445,18.251712,6.370403,0.000055,10.186735,17.916752,17.887634,6.143069,0.000045,8.405800
1,KWASEATT2336,120,13.003032,10.569432,4.833679,0.000026,4.802496,12.554513,10.549833,4.627848,0.000024,4.418152
2,KWASEATT2379,246,10.045915,10.241171,2.289531,0.000016,3.021864,9.946223,10.238841,2.268114,0.000017,3.209244
3,KWASEATT2850,335,7.811290,7.727144,1.478178,0.000013,2.410933,7.820630,7.723146,1.472572,0.000013,2.383573
4,KWASEATT2696,253,6.297961,6.344241,0.996792,0.000007,1.300910,6.575540,6.413519,1.132371,0.000008,1.527417
...,...,...,...,...,...,...,...,...,...,...,...,...
141,KWASEATT1987,269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,KWASEATT2865,213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,KWASEATT2752,82,4.624536,4.430330,1.298633,0.000005,0.971991,4.616506,4.392654,1.291006,0.000005,0.952169
144,KWASEATT2615,46,4.811126,4.751829,0.686310,0.000004,0.789196,4.815838,4.791445,0.678137,0.000004,0.781013


## terrain

In [ ]:
import numpy as np
import rasterio
from rasterio.features import geometry_mask

def terrain_stats(terrain, hexagon):
    # Make sure 'terrain' is a Rasterio dataset and 'hexagon' is the geometry
    geom_mask = geometry_mask([hexagon], transform=terrain.transform, invert=True, out_shape=(terrain.height, terrain.width))

    # Use the mask to select the terrain data
    selected_terrain = terrain.read(1)[geom_mask]

    # Calculate the statistics for the selected terrain data
    if selected_terrain.size > 0:
        mean = np.mean(selected_terrain)
        median = np.median(selected_terrain)
        std_dev = np.std(selected_terrain)
        min_val = np.min(selected_terrain)
        percentile_25 = np.percentile(selected_terrain, 25)
        percentile_50 = np.percentile(selected_terrain, 50)
        percentile_75 = np.percentile(selected_terrain, 75)
        max_val = np.max(selected_terrain)
    else:
        mean = median = std_dev = min_val = percentile_25 = percentile_50 = percentile_75 = max_val = np.nan

    return {
        'terrain_stat_mean': mean,
        'terrain_stat_median': median,
        'terrain_stat_std_dev': std_dev,
        'terrain_stat_min': min_val,
        'terrain_stat_25%': percentile_25,
        'terrain_stat_50%': percentile_50,
        'terrain_stat_75%': percentile_75,
        'terrain_stat_max': max_val
    }

In [ ]:
with rasterio.open(raw_terrain_path) as terrain_dataset:
    for stat in ['mean', 'median', 'std_dev', 'min', '25%','50%', '75%', 'max']:
        raw_hex['terrain_stat_' + stat] = raw_hex.apply(lambda x: terrain_stats(terrain_dataset, x['hex'])[f'terrain_stat_{stat}'], axis=1)

In [ ]:
raw_hex['Elev (m.)'] = raw_hex['Elev (ft.)'].replace('-', np.nan).astype(float) * 0.3048

In [ ]:
raw_hex[['Station ID','Ave temp annual_F', 'Elev (m.)',
          'terrain_stat_mean','terrain_stat_median', 'terrain_stat_std_dev', 'terrain_stat_min', 'terrain_stat_max',
          'centroid_stat_mean','centroid_stat_median','centroid_stat_std_dev', 'centroid_stat_total_height_area', 'centroid_stat_avg_height_area',
          'intercestion_stat_mean','intercestion_stat_median', 'intercestion_stat_std_dev', 'intercestion_stat_total_height_area', 'intercestion_stat_avg_height_area',
       ]]

,Station ID,Ave temp annual_F,Elev (m.),terrain_stat_mean,terrain_stat_median,terrain_stat_std_dev,terrain_stat_min,terrain_stat_max,centroid_stat_mean,centroid_stat_median,centroid_stat_std_dev,centroid_stat_total_height_area,centroid_stat_avg_height_area,intercestion_stat_mean,intercestion_stat_median,intercestion_stat_std_dev,intercestion_stat_total_height_area,intercestion_stat_avg_height_area
0,KWASEATT2119,50.107945,59.1312,44.077553,44.538754,5.613757,16.892885,51.587822,18.714445,18.251712,6.370403,0.000055,10.186735,17.916752,17.887634,6.143069,0.000045,8.405800
1,KWASEATT2336,53.560274,36.5760,28.776283,28.764742,3.721445,20.485500,35.644684,13.003032,10.569432,4.833679,0.000026,4.802496,12.554513,10.549833,4.627848,0.000024,4.418152
2,KWASEATT2379,52.952329,74.9808,71.785751,74.624756,7.484453,49.151745,83.389259,10.045915,10.241171,2.289531,0.000016,3.021864,9.946223,10.238841,2.268114,0.000017,3.209244
3,KWASEATT2850,NaN,102.1080,107.779510,107.553947,3.508487,102.296387,114.796669,7.811290,7.727144,1.478178,0.000013,2.410933,7.820630,7.723146,1.472572,0.000013,2.383573
4,KWASEATT2696,53.064187,77.1144,74.429810,74.360405,4.839781,64.390907,83.490387,6.297961,6.344241,0.996792,0.000007,1.300910,6.575540,6.413519,1.132371,0.000008,1.527417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,KWASEATT1987,NaN,81.9912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,KWASEATT2865,NaN,64.9224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,KWASEATT2752,51.494521,24.9936,24.115768,22.711025,4.817883,17.687239,34.984173,4.624536,4.430330,1.298633,0.000005,0.971991,4.616506,4.392654,1.291006,0.000005,0.952169
144,KWASEATT2615,51.895342,14.0208,13.385434,13.262436,1.905648,9.717768,18.942366,4.811126,4.751829,0.686310,0.000004,0.789196,4.815838,4.791445,0.678137,0.000004,0.781013


## save to drive

In [ ]:
raw_hex.columns

Index(['hex', 'Station ID', 'Lat', 'Lon', 'Elev (ft.)', 'Ave temp annual_F',
       'centroid_stat_ALL', 'intercestion_stat_ALL',
       'centroid_stat_total_height_area', 'centroid_stat_avg_height_area',
       'centroid_stat_mean', 'centroid_stat_median', 'centroid_stat_std_dev',
       'centroid_stat_min', 'centroid_stat_25%', 'centroid_stat_50%',
       'centroid_stat_75%', 'centroid_stat_max',
       'intercestion_stat_total_height_area',
       'intercestion_stat_avg_height_area', 'intercestion_stat_mean',
       'intercestion_stat_median', 'intercestion_stat_std_dev',
       'intercestion_stat_min', 'intercestion_stat_25%',
       'intercestion_stat_50%', 'intercestion_stat_75%',
       'intercestion_stat_max', 'terrain_stat_mean', 'terrain_stat_median',
       'terrain_stat_std_dev', 'terrain_stat_min', 'terrain_stat_25%',
       'terrain_stat_50%', 'terrain_stat_75%', 'terrain_stat_max',
       'Elev (m.)'],
      dtype='object')

In [ ]:
len(raw_hex['Station ID'].unique().tolist())

143

In [ ]:
raw_weather

,Station ID,Station Name,Lat,Lon,Elev (ft.),Ave temp annual_F,Note,hex
0,KWASEATT2119,Belltown - Lenora,47.613,-122.342,194,50.107945,NaN,"POLYGON ((-122.34056108543052 47.613, -122.341..."
1,KWASEATT2336,Avalon LQA Seattle,47.620,-122.356,120,53.560274,NaN,"POLYGON ((-122.35456108543053 47.62, -122.3552..."
2,KWASEATT2379,J'Oz 4 - The Veduta,47.621,-122.327,246,52.952329,NaN,POLYGON ((-122.32556108543052 47.6210000000000...
3,KWASEATT2850,mrweather Capitol Hill,47.622,-122.318,335,NaN,no data,"POLYGON ((-122.31656108543052 47.622, -122.317..."
4,KWASEATT2696,Seattle CD,47.602,-122.312,253,53.064187,NaN,"POLYGON ((-122.31056108543052 47.602, -122.311..."
...,...,...,...,...,...,...,...,...
141,KWASEATT1987,North Beach,47.696,-122.401,269,NaN,NaN,"POLYGON ((-122.39956108543053 47.696, -122.400..."
142,KWASEATT2865,Forest Hill,47.694,-122.398,213,NaN,NaN,POLYGON ((-122.39656108543052 47.6940000000000...
143,KWASEATT2752,Meadowbrook Harriet,47.709,-122.288,82,51.494521,NaN,"POLYGON ((-122.28656108543053 47.709, -122.287..."
144,KWASEATT2615,Matthews Beach 6162,47.700,-122.280,46,51.895342,NaN,"POLYGON ((-122.27856108543052 47.7, -122.27928..."


In [ ]:
raw_hex

,hex,Station ID,Lat,Lon,Elev (ft.),Ave temp annual_F,centroid_stat_ALL,intercestion_stat_ALL,centroid_stat_total_height_area,centroid_stat_avg_height_area,...,intercestion_stat_max,terrain_stat_mean,terrain_stat_median,terrain_stat_std_dev,terrain_stat_min,terrain_stat_25%,terrain_stat_50%,terrain_stat_75%,terrain_stat_max,Elev (m.)
0,"POLYGON ((-122.34056108543052 47.613, -122.341...",KWASEATT2119,47.613,-122.342,194,50.107945,{'centroid_stat_total_height_area': 5.47970141...,{'intercestion_stat_total_height_area': 4.5216...,0.000055,10.186735,...,28.617554,44.077553,44.538754,5.613757,16.892885,42.262379,44.538754,47.429897,51.587822,59.1312
1,"POLYGON ((-122.35456108543053 47.62, -122.3552...",KWASEATT2336,47.620,-122.356,120,53.560274,{'centroid_stat_total_height_area': 2.58338361...,{'intercestion_stat_total_height_area': 2.3766...,0.000026,4.802496,...,21.074318,28.776283,28.764742,3.721445,20.485500,25.851363,28.764742,31.914480,35.644684,36.5760
2,POLYGON ((-122.32556108543052 47.6210000000000...,KWASEATT2379,47.621,-122.327,246,52.952329,{'centroid_stat_total_height_area': 1.62553685...,{'intercestion_stat_total_height_area': 1.7263...,0.000016,3.021864,...,15.485346,71.785751,74.624756,7.484453,49.151745,67.482391,74.624756,77.246803,83.389259,74.9808
3,"POLYGON ((-122.31656108543052 47.622, -122.317...",KWASEATT2850,47.622,-122.318,335,NaN,{'centroid_stat_total_height_area': 1.29690154...,{'intercestion_stat_total_height_area': 1.2821...,0.000013,2.410933,...,11.100597,107.779510,107.553947,3.508487,102.296387,104.684982,107.553947,110.639557,114.796669,102.1080
4,"POLYGON ((-122.31056108543052 47.602, -122.311...",KWASEATT2696,47.602,-122.312,253,53.064187,{'centroid_stat_total_height_area': 6.99792175...,{'intercestion_stat_total_height_area': 8.2163...,0.000007,1.300910,...,8.845202,74.429810,74.360405,4.839781,64.390907,70.521454,74.360405,78.539589,83.490387,77.1144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,"POLYGON ((-122.39956108543053 47.696, -122.400...",KWASEATT1987,47.696,-122.401,269,NaN,"{'centroid_stat_total_height_area': nan, 'cent...","{'intercestion_stat_total_height_area': nan, '...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.9912
142,POLYGON ((-122.39656108543052 47.6940000000000...,KWASEATT2865,47.694,-122.398,213,NaN,"{'centroid_stat_total_height_area': nan, 'cent...","{'intercestion_stat_total_height_area': nan, '...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.9224
143,"POLYGON ((-122.28656108543053 47.709, -122.287...",KWASEATT2752,47.709,-122.288,82,51.494521,{'centroid_stat_total_height_area': 5.22858244...,{'intercestion_stat_total_height_area': 5.1219...,0.000005,0.971991,...,7.789196,24.115768,22.711025,4.817883,17.687239,20.245832,22.711025,27.923845,34.984173,24.9936
144,"POLYGON ((-122.27856108543052 47.7, -122.27928...",KWASEATT2615,47.700,-122.280,46,51.895342,{'centroid_stat_total_height_area': 4.24528604...,{'intercestion_stat_total_height_area': 4.2012...,0.000004,0.789196,...,5.963899,13.385434,13.262436,1.905648,9.717768,11.901059,13.262436,14.375273,18.942366,14.0208


In [ ]:
# raw_hex.merge(raw_weather[['Station ID','Note']], on = 'Station ID', how = 'left')
raw_hex[['Note']] = raw_weather[['Note']]

In [ ]:
raw_weather.columns

Index(['Station ID', 'Station Name', 'Lat', 'Lon', 'Elev (ft.)',
       'Ave temp annual_F', 'Note', 'hex'],
      dtype='object')

In [ ]:
aggr_hex.crs

In [ ]:
raw_boundary.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
aggr_hex = raw_hex[['hex', 'Station ID', 'Lat', 'Lon', 'Elev (ft.)','Elev (m.)',  'Ave temp annual_F', 'Note', 'centroid_stat_ALL', 'intercestion_stat_ALL',

          'centroid_stat_total_height_area', 'centroid_stat_avg_height_area', 'centroid_stat_mean', 'centroid_stat_median', 'centroid_stat_std_dev',
          'centroid_stat_min', 'centroid_stat_25%', 'centroid_stat_50%', 'centroid_stat_75%', 'centroid_stat_max',

          'intercestion_stat_total_height_area','intercestion_stat_avg_height_area', 'intercestion_stat_mean', 'intercestion_stat_median', 'intercestion_stat_std_dev',
          'intercestion_stat_min', 'intercestion_stat_25%', 'intercestion_stat_50%', 'intercestion_stat_75%', 'intercestion_stat_max',

          'terrain_stat_mean','terrain_stat_median', 'terrain_stat_std_dev', 'terrain_stat_min', 'terrain_stat_25%', 'terrain_stat_50%', 'terrain_stat_75%', 'terrain_stat_max',]]

In [ ]:
raw_hex = gpd.GeoDataFrame(aggr_hex, geometry = 'hex')

raw_hex.to_file(aggr_hex_path, driver = "GeoJSON")

In [ ]:
# aggr_hex = gpd.GeoDataFrame(aggr_hex, geometry = 'hex', crs = "EPSG:4326")
aggr_hex = gpd.GeoDataFrame(aggr_hex, geometry = 'hex')

aggr_hex.to_file(aggr_hex_path, driver = "GeoJSON")

In [ ]:
m = folium.Map(location=[aggr_hex.iloc[0]['Lat'], aggr_hex.iloc[0]['Lon']], zoom_start=14)
for _, row in aggr_hex.iterrows():
    geojson_hex = hex_to_geojson(row['hex'])
    geojson_hex.add_to(m)
    # Add the point
    folium.Marker(
        [row['Lat'], row['Lon']],
        popup=f'Lat: {row["Lat"]}, Lon: {row["Lon"]}'
    ).add_to(m)
m